<a href="https://colab.research.google.com/github/robin-tanr/Agnos_Robin/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from pathlib import Path
import os
PROJ = Path(os.getenv("PROJ", "/content/drive/MyDrive/AgnosHealth"))
if not (PROJ / "AgnosDataset.csv").exists():
    PROJ = Path(".").resolve()  # fallback to repo root
print("Project root:", PROJ)

Project root: /content/drive/MyDrive/AgnosHealth
CSV exists: True -> /content/drive/MyDrive/AgnosHealth/AgnosDataset.csv
Subfolders: ['data', 'figures', 'notebooks', 'src']


In [ ]:
import pandas as pd, json, ast, re

CSV_PATH = PROJ / "AgnosDataset.csv"
assert CSV_PATH.exists(), f"ไม่พบไฟล์: {CSV_PATH}"

read_ok = False
for enc in ["utf-8", "utf-8-sig", "cp874", "latin-1"]:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc, keep_default_na=False, low_memory=False)
        read_ok = True
        used_enc = enc
        break
    except Exception:
        pass
if not read_ok:
    raise RuntimeError("อ่าน CSV ไม่สำเร็จ ลองตรวจ encoding/รูปแบบไฟล์อีกครั้ง")
print(f"Rows: {len(df)}  |  Encoding: {used_enc}")
print("Original columns:", list(df.columns))

def _canon(s):
    return re.sub(r"[\s_]+", "", str(s).strip().lower())

def find_col(candidates, default=None):
    cand = {_canon(c) for c in candidates}
    for col in df.columns:
        if _canon(col) in cand:
            return col
    return default

sex_col     = find_col(["sex","gender","เพศ"])
age_col     = find_col(["age","อายุ","age_years","age_yrs"])
search_col  = find_col(["search_term","search","query","keyword","term","อาการที่ค้นหา"])
summary_col = find_col(["summary","symptoms_json","symptom_json","json","detail","symptoms","ข้อมูลสรุป"])

if summary_col is None:
    for col in df.columns:
        v = str(df[col].iloc[0])
        if ("{" in v and "}" in v) or ("[" in v and "]" in v):
            summary_col = col
            break

# standardize
df["sex"] = df[sex_col] if sex_col else ""
df["age"] = df[age_col] if age_col else ""
df["search_term"] = df[search_col] if search_col else ""

if summary_col is None:
    raise ValueError("ไม่พบคอลัมน์ summary (JSON) — ต้องมี keys: yes_symptoms / no_symptoms / idk_symptoms ตามสเปค Task 2")

#  parse summary
def _as_list(v):
    if v is None: return []
    if isinstance(v, str): return [v] if v.strip() else []
    if isinstance(v, (list, tuple, set)):
        return [x for x in v if isinstance(x, str) and x.strip()]
    return []

def _safe_parse_summary(s):
    if isinstance(s, dict):
        d = s
    elif isinstance(s, str):
        t = s.strip()
        if not t:
            d = {}
        else:
            try:
                d = json.loads(t)
            except Exception:
                try:
                    d = ast.literal_eval(t)
                except Exception:
                    d = {}
    else:
        d = {}

    # alias ของ key
    def _get(d, keys):
        for k in keys:
            if k in d: return d[k]
        return []
    yes = _as_list(_get(d, ["yes_symptoms","yes","symptoms_yes"]))
    no  = _as_list(_get(d, ["no_symptoms","no","symptoms_no"]))
    idk = _as_list(_get(d, ["idk_symptoms","idk","unknown","unsure"]))
    return {"yes": yes, "no": no, "idk": idk}

parsed = df[summary_col].apply(_safe_parse_summary)
df["yes_raw"] = parsed.apply(lambda x: x["yes"])
df["no_raw"]  = parsed.apply(lambda x: x["no"])
df["idk_raw"] = parsed.apply(lambda x: x["idk"])

# preview
print("\nMapped columns ->",
      {"sex": sex_col, "age": age_col, "search_term": search_col, "summary": summary_col})

print(df[["sex","age","search_term"]].head(3))

print("\nLens (first 5 rows):")
print(pd.DataFrame({
    "yes_len": df["yes_raw"].str.len(),
    "no_len":  df["no_raw"].str.len(),
    "idk_len": df["idk_raw"].str.len(),
}).head())

Rows: 1000  |  Encoding: utf-8
Original columns: ['gender', 'age', 'summary', 'search_term']

Mapped columns -> {'sex': 'gender', 'age': 'age', 'search_term': 'search_term', 'summary': 'summary'}
      sex  age    search_term
0    male   28    มีเสมหะ, ไอ
1    male   27  ไอ, น้ำมูกไหล
2  female   26        ปวดท้อง

Lens (first 5 rows):
   yes_len  no_len  idk_len
0        0       0        0
1        0       0        0
2        0       0        0
3        0       0        0
4        0       0        0


In [ ]:
import re

col = "summary"

lens = df[col].astype(str).str.len()
has_brace = df[col].astype(str).str.contains(r"[{}\[\]]")
print("summary non-empty rows:", int((lens > 0).sum()))
print("rows that look JSON-like (has {} or []):", int(has_brace.sum()))

print("\n--- 5 raw examples (repr) ---")
for i, s in enumerate(df[col].astype(str).head(5), 1):
    print(f"{i}: {repr(s)[:240]}")

summary non-empty rows: 1000
rows that look JSON-like (has {} or []): 1000

--- 5 raw examples (repr) ---
1: '{"diseases": [], "procedures": [], "no_symptoms": [], "idk_symptoms": [], "yes_symptoms": [{"text": "เสมหะ", "answers": ["ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว"]}, {"text": "ไอ", "answers": ["ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)"]}, {"te
2: '{"diseases": [], "procedures": [], "no_symptoms": [], "idk_symptoms": [], "yes_symptoms": [{"text": "ไอ", "answers": ["ระยะเวลา 1-3 สัปดาห์", "ลักษณะ ไอไม่มีเสมหะ ไอแห้งๆ หรือไอเสมหะสีขาว"]}, {"text": "น้ำมูกไหล", "answers": ["ระยะเวลา 10 
3: '{"diseases": [], "procedures": [], "no_symptoms": [], "idk_symptoms": [], "yes_symptoms": [{"text": "ปวดท้อง", "answers": ["บริเวณ รอบๆสะดือ", "ระยะเวลา ตั้งแต่ 1 วัน ถึง 1 สัปดาห์", "ระดับ ปวดจนไม่สามารถทำงานได้"]}, {"text": "การรักษาก่อน
4: '{"diseases": [], "procedures": [], "no_symptoms": [], "idk_symptoms": [], "yes_symptoms": [{"text": "น้ำมูกไหล", "answers": ["ระยะเวลา น้อยกว่า 10 วัน", "ประว

In [ ]:
import json, ast

def parse_nested_json_like(x, max_depth=3):
    """Try to unwrap nested/escaped JSON or Python-literal up to max_depth."""
    if isinstance(x, (dict, list, tuple, set)):
        return x
    s = x
    for _ in range(max_depth):
        if isinstance(s, (dict, list, tuple, set)):
            return s
        if not isinstance(s, str):
            break
        t = s.strip()
        if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
            t = t[1:-1].strip()
        try:
            s = json.loads(t); continue
        except Exception:
            pass
        try:
            s = ast.literal_eval(t); continue
        except Exception:
            pass
        break
    return s

def _as_list_str(v):
    """Return *string* list from mixed types."""
    if v is None:
        return []
    out = []
    if isinstance(v, str):
        if v.strip():
            out.append(v.strip())
    elif isinstance(v, (list, tuple, set)):
        for item in v:
            if isinstance(item, str) and item.strip():
                out.append(item.strip())
            elif isinstance(item, dict):
                # try common keys carrying the symptom string
                for k in ("text", "symptom", "name", "label", "title"):
                    if k in item and isinstance(item[k], str) and item[k].strip():
                        out.append(item[k].strip())
                        break
    elif isinstance(v, dict):
        # occasionally a single dict holding a symptom under a key
        for k in ("text", "symptom", "name", "label", "title"):
            if k in v and isinstance(v[k], str) and v[k].strip():
                out.append(v[k].strip())
                break
    return out

def parse_summary_robust(val):
    obj = parse_nested_json_like(val, max_depth=4)

    # sometimes list of dicts wrapped in a list (we keep as-is; _as_list_str handles dicts)
    if isinstance(obj, list) and obj and isinstance(obj[0], dict) and \
       all(isinstance(d, dict) for d in obj):
        # not merging; we'll extract strings from the target keys directly below
        pass

    # if dict at top-level, pull keys; else leave empty
    yes = no = idk = []
    if isinstance(obj, dict):
        # prefer exact names per Task 2, with fallbacks
        yes_src = obj.get("yes_symptoms") or obj.get("yes") or obj.get("symptoms_yes")
        no_src  = obj.get("no_symptoms")  or obj.get("no")  or obj.get("symptoms_no")
        idk_src = obj.get("idk_symptoms") or obj.get("idk") or obj.get("unknown") or obj.get("unsure")
        yes = _as_list_str(yes_src)
        no  = _as_list_str(no_src)
        idk = _as_list_str(idk_src)
    else:
        # unexpected shape → empty
        yes, no, idk = [], [], []

    return {"yes": yes, "no": no, "idk": idk}

# rewrite raw columns
parsed2 = df["summary"].apply(parse_summary_robust)
df["yes_raw"] = parsed2.apply(lambda x: x["yes"])
df["no_raw"]  = parsed2.apply(lambda x: x["no"])
df["idk_raw"] = parsed2.apply(lambda x: x["idk"])

print("Non-empty yes rows:", int((df['yes_raw'].str.len() > 0).sum()))
print("Non-empty no rows:",  int((df['no_raw'].str.len()  > 0).sum()))
print("Non-empty idk rows:", int((df['idk_raw'].str.len() > 0).sum()))

print("\nExamples (first 3 rows):")
print(df[["yes_raw","no_raw","idk_raw"]].head(3))

Non-empty yes rows: 1000
Non-empty no rows: 79
Non-empty idk rows: 20

Examples (first 3 rows):
                             yes_raw no_raw idk_raw
0      [เสมหะ, ไอ, การรักษาก่อนหน้า]     []      []
1  [ไอ, น้ำมูกไหล, การรักษาก่อนหน้า]     []      []
2        [ปวดท้อง, การรักษาก่อนหน้า]     []      []


In [ ]:
import re, json
from pathlib import Path

SYM_MAP_PATH = PROJ / "symptom_map.json"
STOP_PATH    = PROJ / "stop_terms.txt"

# defaults
SYM_MAP_DEFAULT = {
  "canonical": {
    # EN → TH
    "cough": "ไอ",
    "night cough": "ไอกลางคืน",
    "fever": "ไข้",
    "chills": "หนาวสั่น",
    "sore throat": "เจ็บคอ",
    "runny nose": "น้ำมูกไหล",
    "stuffy nose": "คัดจมูก",
    "sneezing": "จาม",
    "wheezing": "หายใจมีเสียงหวีด",
    "shortness of breath": "หายใจลำบาก",
    "labored breathing": "หายใจลำบาก",
    "difficulty breathing": "หายใจลำบาก",
    "headache": "ปวดหัว",
    "migraine": "ไมเกรน",
    "chest pain": "เจ็บหน้าอก",
    "fatigue": "อ่อนเพลีย",
    "nausea": "คลื่นไส้",
    "vomiting": "อาเจียน",
    "diarrhea": "ท้องเสีย",
    "constipation": "ท้องผูก",
    "abdominal pain": "ปวดท้อง",
    "back pain": "ปวดหลัง",
    "muscle pain": "ปวดกล้ามเนื้อ",
    "joint pain": "ปวดข้อ",
    "dizziness": "เวียนศีรษะ",
    "rash": "ผื่น",
    "itchy": "คัน",
    "itching": "คัน"
  },
  "thai_alias": {
    # TH variants → TH canonical
    "ไอมีเสมหะ": "ไอ",
    "ไอแห้ง": "ไอ",
    "แน่นจมูก": "คัดจมูก",
    "เหนื่อยหอบ": "หายใจลำบาก",
    "แน่นหน้าอก": "เจ็บหน้าอก",
    "ปวดท้ายทอย": "ปวดหัว"
  }
}
STOP_DEFAULT = [
  "ประวัติอุบัติเหตุ","การรักษาก่อนหน้า","แพ้ยา","โรคประจำตัว",
  "ตั้งครรภ์","ผ่าตัด","วัคซีน","ประวัติครอบครัว",
  "ประวัติการสูบบุหรี่","ประวัติการดื่มแอลกอฮอล์"
]

SYM_MAP = SYM_MAP_DEFAULT
if SYM_MAP_PATH.exists():
    try:
        SYM_MAP = json.loads(SYM_MAP_PATH.read_text(encoding="utf-8"))
        print("Loaded symptom_map.json from Drive")
    except Exception as e:
        print("WARN: symptom_map.json invalid, using default:", e)

STOP_TERMS = set(STOP_DEFAULT)
if STOP_PATH.exists():
    try:
        STOP_TERMS = set([s.strip() for s in STOP_PATH.read_text(encoding="utf-8").splitlines() if s.strip()])
        print("Loaded stop_terms.txt from Drive")
    except Exception as e:
        print("WARN: stop_terms.txt invalid, using default:", e)

COMPOUNDS = {
    "น้ำมูกไหลไอ": ["น้ำมูกไหล", "ไอ"],
    "ปวดหัวปวดท้ายทอย": ["ปวดหัว", "ปวดท้ายทอย"],
    "คัดจมูกน้ำมูกไหล": ["คัดจมูก", "น้ำมูกไหล"],
}

# connectors / prefixes
CONNECTOR_RE = r"\s*(?:,|\+|\||และ|กับ|ร่วมกับ)\s*"
PREFIX_RE    = r"^(?:มี|เป็น|รู้สึก|รู้สึกว่า)\s*"

def to_canonical_th(token: str) -> str:
    t = (token or "").strip().lower()
    if not t: return ""
    # EN→TH
    if t in SYM_MAP.get("canonical", {}):
        t = SYM_MAP["canonical"][t]
    # TH variants→TH canonical
    t = SYM_MAP.get("thai_alias", {}).get(t, t)
    # ตัด prefixทั่วไป
    t = re.sub(PREFIX_RE, "", t).strip()
    return t

def split_compounds(text: str):
    text = (text or "").strip()
    if not text: return []
    # แยกด้วย connectors (, + | และ/กับ/ร่วมกับ)
    parts = re.split(CONNECTOR_RE, text)
    out = []
    for p in parts:
        p = re.sub(PREFIX_RE, "", p).strip()
        if not p: continue
        if p in COMPOUNDS:
            out.extend(COMPOUNDS[p])
        else:
            out.append(p)
    return out

def normalize_list(tokens):
    out, seen = [], set()
    for tok in (tokens or []):
        for t in split_compounds(tok):
            t = to_canonical_th(t)
            if t and (t not in STOP_TERMS) and (t not in seen):
                out.append(t); seen.add(t)
    return out

def normalize_free_text(text):
    return normalize_list([text])

def normalize_free_text_multi(text):
    """รองรับ search_term ที่คั่นด้วย , + | และ/กับ/ร่วมกับ → list"""
    return normalize_list([text])

print("Normalizer ready. Example:",
      normalize_list(["น้ำมูกไหลไอ","ปวดหัวปวดท้ายทอย","มีคัดจมูก","wheezing","shortness of breath"]))

Normalizer ready. Example: ['น้ำมูกไหล', 'ไอ', 'ปวดหัว', 'คัดจมูก', 'หายใจมีเสียงหวีด', 'หายใจลำบาก']


In [ ]:
def _norm_sex(x):
    s = str(x).strip().lower()
    if s in {"m","male","ชาย","boy","man"}: return "M"
    if s in {"f","female","หญิง","girl","woman"}: return "F"
    return "U"

def _age_band(a):
    try:
        a = int(float(a))
    except:
        return "20-40"
    if a <= 12: return "0-12"
    if a <= 19: return "13-19"
    if a <= 40: return "20-40"
    if a <= 60: return "41-60"
    return "60+"

# normalize lists
df["basket"]   = df["yes_raw"].apply(normalize_list)
df["no_norm"]  = df["no_raw"].apply(normalize_list)
df["idk_norm"] = df["idk_raw"].apply(normalize_list)
df["basket_len"] = df["basket"].str.len()

# filter out empty baskets
before, after = len(df), int((df["basket_len"] > 0).sum())
df = df[df["basket_len"] > 0].reset_index(drop=True)

# sex/age/segment
df["sex_"]     = df["sex"].apply(_norm_sex)
df["age_band"] = df["age"].apply(_age_band)
df["seg"]      = df["sex_"] + "_" + df["age_band"]

# search_term → list (เช่น "มีเสมหะ, ไอ" → ["เสมหะ","ไอ"])
df["search_norm"] = df["search_term"].apply(normalize_free_text_multi)

print(f"Rows before: {before}  |  after basket-filter: {len(df)}")
print("Segments (top 10):")
print(df["seg"].value_counts().head(10))

print("\nPreview (first 3 rows):")
print(df[["yes_raw","basket","search_term","search_norm"]].head(3))

Rows before: 1000  |  after basket-filter: 991
Segments (top 10):
seg
F_20-40    351
M_20-40    213
F_41-60    174
M_41-60    137
F_60+       50
M_60+       28
M_13-19     13
F_13-19     12
F_0-12       8
M_0-12       5
Name: count, dtype: int64

Preview (first 3 rows):
                             yes_raw           basket    search_term  \
0      [เสมหะ, ไอ, การรักษาก่อนหน้า]      [เสมหะ, ไอ]    มีเสมหะ, ไอ   
1  [ไอ, น้ำมูกไหล, การรักษาก่อนหน้า]  [ไอ, น้ำมูกไหล]  ไอ, น้ำมูกไหล   
2        [ปวดท้อง, การรักษาก่อนหน้า]        [ปวดท้อง]        ปวดท้อง   

       search_norm  
0      [เสมหะ, ไอ]  
1  [ไอ, น้ำมูกไหล]  
2        [ปวดท้อง]  


In [ ]:
from collections import Counter

def _pop_from_baskets(series_of_lists):
    c = Counter()
    for lst in series_of_lists:
        c.update(lst)
    return c

global_pop = _pop_from_baskets(df["basket"])

by_seg = {}
for seg, sub in df.groupby("seg"):
    by_seg[seg] = _pop_from_baskets(sub["basket"])

print("Global top-15:")
for s, c in global_pop.most_common(15):
    print(f"{s:20s}  {c}")

print("\nSample 3 segments (top-8 each):")
for seg in list(sorted(by_seg.keys()))[:3]:
    print(f"\n[{seg}]")
    for s, c in by_seg[seg].most_common(8):
        print(f"  {s:20s}  {c}")

Global top-15:
ไอ                    202
เจ็บคอ                124
เสมหะ                 116
ไข้                   109
น้ำมูกไหล             102
previous treatment    81
ปวดข้อ                74
ปวดท้อง               63
ผื่น                  49
คัดจมูก               43
ท้องเสีย              41
คัน                   31
เวียนศีรษะ บ้านหมุน   28
ปวดหัว                19
ปวดเข่า               17

Sample 3 segments (top-8 each):

[F_0-12]
  previous treatment    4
  ไข้                   3
  ไอ                    1
  ปวดท้อง               1
  อาเจียน               1
  skin rash             1
  เจ็บคอ                1

[F_13-19]
  ปวดท้อง               2
  ผื่น                  2
  ไข้                   2
  previous treatment    2
  เสมหะ                 1
  เจ็บคอ                1
  กลืนเจ็บ              1
  ไอ                    1

[F_20-40]
  ไอ                    87
  เสมหะ                 51
  เจ็บคอ                47
  น้ำมูกไหล             41
  ไข้                   38
  ปวดท้อง      

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

transactions = df["basket"].tolist()

# one-hot
te = TransactionEncoder()
oht = te.fit_transform(transactions)
oht_df = pd.DataFrame(oht, columns=te.columns_)


min_sup = max(5 / len(df), 0.005)

freq = fpgrowth(oht_df, min_support=min_sup, use_colnames=True)
print(f"Frequent itemsets: {len(freq)} (min_support={min_sup:.4f})")

rules = association_rules(freq, metric="lift", min_threshold=1.0)

rules = rules[(rules["antecedents"].map(len)==1) & (rules["consequents"].map(len)==1)].copy()

rules["A"] = rules["antecedents"].apply(lambda s: next(iter(s)))
rules["B"] = rules["consequents"].apply(lambda s: next(iter(s)))
rules["rank_score"] = rules["lift"] * rules["confidence"]
rules = rules.sort_values(["rank_score","lift","confidence"], ascending=False)

print(f"Rules (1⇒1): {len(rules)}")

rule_index = {}
for _, r in rules.iterrows():
    A, B = r["A"], r["B"]
    rule_index.setdefault(A, {})[B] = {
        "lift": float(r["lift"]),
        "confidence": float(r["confidence"]),
        "support": float(r["support"])
    }

print("\nTop-10 rules (A ⇒ B):")
for _, r in rules.head(10).iterrows():
    print(f"{r['A']} ⇒ {r['B']} | supp {r['support']:.3f}  conf {r['confidence']:.3f}  lift {r['lift']:.2f}")

Frequent itemsets: 77 (min_support=0.0050)
Rules (1⇒1): 44

Top-10 rules (A ⇒ B):
เดินเซ ทรงตัวไม่ได้ ⇒ เวียนศีรษะ บ้านหมุน | supp 0.005  conf 1.000  lift 35.39
ปวดไหล่ ⇒ ปวดต้นคอ | supp 0.005  conf 0.455  lift 37.54
ปวดต้นคอ ⇒ ปวดไหล่ | supp 0.005  conf 0.417  lift 37.54
ปวดข้อเท้า ⇒ ปวดข้อ | supp 0.007  conf 1.000  lift 13.39
ปวดข้อมือ ⇒ ปวดข้อ | supp 0.007  conf 1.000  lift 13.39
ปวดเข่า ⇒ ปวดข้อ | supp 0.017  conf 1.000  lift 13.39
ปวดไหล่ ⇒ ปวดข้อ | supp 0.011  conf 1.000  lift 13.39
skin rash ⇒ previous treatment | supp 0.008  conf 1.000  lift 12.23
itch ⇒ previous treatment | supp 0.006  conf 1.000  lift 12.23
คัน ⇒ ผื่น | supp 0.022  conf 0.710  lift 14.35


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
ADMIN_NON_SYMPTOMS = {"previous treatment"}
if ADMIN_NON_SYMPTOMS:
    df["basket"] = df["basket"].apply(lambda lst: [x for x in lst if x not in ADMIN_NON_SYMPTOMS])

from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

transactions = df["basket"].tolist()

te = TransactionEncoder()
oht = te.fit_transform(transactions)
oht_df = pd.DataFrame(oht, columns=te.columns_)

#  item×item
X = oht.astype(np.float32)               # (n_rows, n_symptoms)
SIM = cosine_similarity(X.T)             # (n_symptoms, n_symptoms)

symptoms = list(te.columns_)
sym2idx  = {s:i for i,s in enumerate(symptoms)}

def cf_score(selected, cand):
    """คะแนน CF = ค่าเฉลี่ย cosine(sim(cand, each selected))."""
    idx = [sym2idx[s] for s in selected if s in sym2idx]
    if not idx or cand not in sym2idx:
        return 0.0
    j = sym2idx[cand]
    return float(SIM[j, idx].mean())

print("CF ready. #symptoms =", len(symptoms))
print("Example cf_score(['ไอ'], 'น้ำมูกไหล') =", cf_score(['ไอ'], 'น้ำมูกไหล'))

CF ready. #symptoms = 142
Example cf_score(['ไอ'], 'น้ำมูกไหล') = 0.30653268098831177


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
ALPHA = 0.7  # ตามผลที่เลือกไว้
K_DEF = 3

def rule_score(selected, cand):
    """คืน (score, best_anchor, best_lift) จาก rule_index (A⇒B)"""
    score = 0.0
    best_a, best_lift = None, 0.0
    for a in selected:
        bmap = rule_index.get(a, {})
        if cand in bmap:
            lift = bmap[cand]["lift"]
            conf = bmap[cand]["confidence"]
            val  = lift * conf
            if val > score: score = val
            if lift > best_lift: best_lift, best_a = lift, a
    return score, best_a, best_lift

def _norm_sex(x):
    s = str(x).strip().lower()
    if s in {"m","male","ชาย","boy","man"}: return "M"
    if s in {"f","female","หญิง","girl","woman"}: return "F"
    return "U"

def _age_band(a):
    try:
        a = int(float(a))
    except:
        return "20-40"
    if a <= 12: return "0-12"
    if a <= 19: return "13-19"
    if a <= 40: return "20-40"
    if a <= 60: return "41-60"
    return "60+"

def recommend(age=None, sex="U", selected=None, top_k=K_DEF, alpha=ALPHA):
    # normalize รายการอาการที่ผู้ใช้เลือก (TH/EN/คั่นด้วย , + | และ/กับ/ร่วมกับ)
    selected = normalize_list(selected or [])
    selected = [s for s in selected if s in symptoms]  # กัน token แปลก

    # cold-start → ใช้ popularity ตาม segment
    if len(selected) == 0:
        seg = f"{_norm_sex(sex)}_{_age_band(age)}"
        pop = by_seg.get(seg, global_pop)
        recs = [{"symptom": s, "score": float(c), "why": "segment popularity"}
                for s, c in pop.most_common(top_k)]
        return selected, recs

    cand = set(symptoms) - set(selected)
    scored = []
    for c in cand:
        rs, anchor, lift = rule_score(selected, c)   # rule-based
        cs = cf_score(selected, c)                   # CF-based
        score = alpha*rs + (1-alpha)*cs
        if anchor:
            why = f"{anchor} ⇒ {c} (lift {lift:.2f})"
        else:
            # ชี้แจง CF ให้เข้าใจง่าย
            why = f"item-CF similar to {', '.join(selected[:1])}"
        scored.append((c, score, why))

    scored.sort(key=lambda x: x[1], reverse=True)
    out = [{"symptom": s, "score": float(sc), "why": w} for s,sc,w in scored[:top_k]]
    return selected, out

tests = [
    dict(age=28, sex="F", selected=["ไอ","น้ำมูกไหล"], top_k=3),
    dict(age=35, sex="M", selected=["cough","stuffy nose"], top_k=5),  # EN input
    dict(age=9,  sex="U", selected=[], top_k=3),                        # cold-start
]

for t in tests:
    inp, recs = recommend(**t)
    print("\nINPUT ->", t)
    print("normalized:", inp)
    for r in recs:
        print("  -", r)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


INPUT -> {'age': 28, 'sex': 'F', 'selected': ['ไอ', 'น้ำมูกไหล'], 'top_k': 3}
normalized: ['ไอ', 'น้ำมูกไหล']
  - {'symptom': 'เสมหะ', 'score': 0.4261034219302778, 'why': 'ไอ ⇒ เสมหะ (lift 2.07)'}
  - {'symptom': 'คัดจมูก', 'score': 0.18897677819801204, 'why': 'น้ำมูกไหล ⇒ คัดจมูก (lift 2.26)'}
  - {'symptom': 'เจ็บคอ', 'score': 0.01688414476811886, 'why': 'item-CF similar to ไอ'}

INPUT -> {'age': 35, 'sex': 'M', 'selected': ['cough', 'stuffy nose'], 'top_k': 5}
normalized: ['ไอ', 'คัดจมูก']
  - {'symptom': 'น้ำมูกไหล', 'score': 0.4364486985218221, 'why': 'คัดจมูก ⇒ น้ำมูกไหล (lift 2.26)'}
  - {'symptom': 'เสมหะ', 'score': 0.4147696404494886, 'why': 'ไอ ⇒ เสมหะ (lift 2.07)'}
  - {'symptom': 'เลือดกำเดาไหล', 'score': 0.03234983235597611, 'why': 'item-CF similar to ไอ'}
  - {'symptom': 'shortness of breath when lying down', 'score': 0.022874785959720614, 'why': 'item-CF similar to ไอ'}
  - {'symptom': 'itchy nose', 'score': 0.022874785959720614, 'why': 'item-CF similar to ไอ'}

INPUT -

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
import warnings, os

warnings.filterwarnings("ignore", category=DeprecationWarning)

warnings.filterwarnings("ignore", category=DeprecationWarning, module=r"jupyter_client.*")

os.environ["PYTHONWARNINGS"] = "ignore::DeprecationWarning"

print("DeprecationWarnings silenced")

DeprecationWarnings silenced


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# Cell — One-shot Evaluation (define + run quietly)
import pandas as pd, warnings, contextlib, io

# === metrics ===
def precision_at_k(recs, gt, k=5):
    if k <= 0: return 0.0
    hit = sum(1 for r in recs[:k] if r["symptom"] in gt)
    return hit / k

def recall_at_k(recs, gt, k=5):
    if not gt: return 0.0
    hit = sum(1 for r in recs[:k] if r["symptom"] in gt)
    return hit / len(gt)

def ap_at_k(recs, gt, k=5):
    if not gt: return 0.0
    s = c = 0.0
    for i, r in enumerate(recs[:k], 1):
        if r["symptom"] in gt:
            c += 1
            s += c / i
    return s / min(len(gt), k)

def coverage(recs_all, universe):
    seen = set()
    for recs in recs_all:
        for r in recs: seen.add(r["symptom"])
    return (len(seen) / len(universe)) if universe else 0.0

# === evaluator ===
ALPHA = globals().get("ALPHA", 0.7)

def eval_all(Ks=(3,5,7,10), alpha=ALPHA):
    assert "df" in globals(), "df not found — รัน Cell 3B→5 ก่อน"
    assert "symptoms" in globals(), "symptoms not found — รัน Cell 8 (CF) ก่อน"
    assert "recommend" in globals(), "recommend() not found — รัน Cell 9 ก่อน"

    rows = []

    # (A) LOO: ใช้เฉพาะ basket≥2, ซ่อนตัวท้าย
    loo = df[df["basket"].str.len() >= 2]
    n_loo = len(loo)
    for K in Ks:
        P = R = AP = 0.0
        recs_all = []
        for _, row in loo.iterrows():
            b = row["basket"]
            hidden, context = b[-1], b[:-1]
            _, recs = recommend(age=row.get("age"), sex=row.get("sex_"),
                                selected=context, top_k=K, alpha=alpha)
            gt = {hidden}
            P += precision_at_k(recs, gt, K)
            R += recall_at_k(recs, gt, K)
            AP += ap_at_k(recs, gt, K)
            recs_all.append(recs)
        rows.append({
            "Proto":"LOO","N":n_loo,"K":K,
            "Precision": P/n_loo if n_loo else 0.0,
            "Recall":    R/n_loo if n_loo else 0.0,
            "MAP":       AP/n_loo if n_loo else 0.0,
            "Coverage":  coverage(recs_all, set(symptoms)),
        })

    # (B) Beyond-search: input = search_norm; GT = basket \ search
    bs = df[df["search_norm"].map(len) > 0]
    for K in Ks:
        P = R = AP = 0.0
        recs_all, cnt = [], 0
        for _, row in bs.iterrows():
            selected = row["search_norm"]
            gt = set(row["basket"]) - set(selected)
            if not gt:
                continue
            _, recs = recommend(age=row.get("age"), sex=row.get("sex_"),
                                selected=selected, top_k=K, alpha=alpha)
            P += precision_at_k(recs, gt, K)
            R += recall_at_k(recs, gt, K)
            AP += ap_at_k(recs, gt, K)
            recs_all.append(recs)
            cnt += 1
        rows.append({
            "Proto":"Beyond","N":cnt,"K":K,
            "Precision": P/cnt if cnt else 0.0,
            "Recall":    R/cnt if cnt else 0.0,
            "MAP":       AP/cnt if cnt else 0.0,
            "Coverage":  coverage(recs_all, set(symptoms)),
        })
    return pd.DataFrame(rows)

buf = io.StringIO()
with contextlib.redirect_stderr(buf), warnings.catch_warnings():
    warnings.simplefilter("ignore", category=DeprecationWarning)
    eval_df = eval_all(alpha=ALPHA)

cols = ["Precision","Recall","MAP","Coverage"]
eval_df_display = eval_df.copy()
eval_df_display[cols] = eval_df_display[cols].round(6)
display(eval_df_display)

# one-liners for slides
def pick(df, proto="LOO", K=5, metric="MAP"):
    row = df[(df["Proto"]==proto) & (df["K"]==K)]
    return (row[metric].iloc[0] if len(row) else float("nan"))

print(f"LOO MAP@5 = {pick(eval_df,'LOO',5,'MAP'):.3f}")
print(f"Beyond MAP@5 = {pick(eval_df,'Beyond',5,'MAP'):.3f}")
print(f"Coverage@5 = {pick(eval_df,'LOO',5,'Coverage'):.3f}")


,Proto,N,K,Precision,Recall,MAP,Coverage
0,LOO,440,3,0.271970,0.815909,0.635227,0.626761
1,LOO,440,5,0.180000,0.900000,0.654318,0.711268
2,LOO,440,7,0.136039,0.952273,0.662273,0.739437
3,LOO,440,10,0.099091,0.990909,0.666673,0.767606
4,Beyond,295,3,0.187571,0.482203,0.393785,0.535211
5,Beyond,295,5,0.132881,0.558475,0.411893,0.633803
6,Beyond,295,7,0.108959,0.626836,0.422573,0.697183
7,Beyond,295,10,0.084068,0.694633,0.430714,0.732394


LOO MAP@5 = 0.654
Beyond MAP@5 = 0.412
Coverage@5 = 0.711


In [ ]:
from pathlib import Path
import json

SYM_PATH = PROJ / "symptom_map.json"
if SYM_PATH.exists():
    mp = json.loads(SYM_PATH.read_text(encoding="utf-8"))
else:
    mp = {"canonical": {}, "thai_alias": {}}

mp.setdefault("canonical", {})
mp.setdefault("thai_alias", {})

add_canonical = {
    # EN → TH (เพิ่มจากที่เห็นในผลแนะนำ/ดาต้า)
    "itchy nose": "คันจมูก",
    "skin rash": "ผื่น",
    "nosebleed": "เลือดกำเดาไหล",
    "bloody nose": "เลือดกำเดาไหล",
    "sputum": "เสมหะ",
    "phlegm": "เสมหะ",
    "rhinorrhea": "น้ำมูกไหล",
    "orthopnea": "หายใจลำบาก",
    "shortness of breath when lying down": "หายใจลำบาก",
    "postnasal drip": "น้ำมูกไหล",
    "night cough": "ไอกลางคืน"
}
add_thai_alias = {
    # TH variants → TH canonical (รวมคำที่เจอบ่อย)
    "เวียนศีรษะ บ้านหมุน": "เวียนศีรษะ",
    "ผื่นคัน": "ผื่น",
    "เลือดกำเดา": "เลือดกำเดาไหล",
    "ไอตอนกลางคืน": "ไอกลางคืน",
    "แน่นจมูก": "คัดจมูก",
    "เหนื่อยหอบ": "หายใจลำบาก",
    "กลืนเจ็บ": "เจ็บคอ"
}

# merge
for k, v in add_canonical.items():
    mp["canonical"].setdefault(k, v)
for k, v in add_thai_alias.items():
    mp["thai_alias"].setdefault(k, v)

SYM_PATH.write_text(json.dumps(mp, ensure_ascii=False, indent=2), encoding="utf-8")
print("Updated:", SYM_PATH)
print("Added canonical:", sorted(add_canonical.keys()))
print("Added thai_alias:", sorted(add_thai_alias.keys()))

Updated: /content/drive/MyDrive/AgnosHealth/symptom_map.json
Added canonical: ['bloody nose', 'itchy nose', 'night cough', 'nosebleed', 'orthopnea', 'phlegm', 'postnasal drip', 'rhinorrhea', 'shortness of breath when lying down', 'skin rash', 'sputum']
Added thai_alias: ['กลืนเจ็บ', 'ผื่นคัน', 'เลือดกำเดา', 'เวียนศีรษะ บ้านหมุน', 'เหนื่อยหอบ', 'แน่นจมูก', 'ไอตอนกลางคืน']


In [ ]:
import json, re
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd, numpy as np

# 1) reload maps/stop terms
SYM_MAP = json.loads((PROJ / "symptom_map.json").read_text(encoding="utf-8"))
STOP_TERMS = set([
  "ประวัติอุบัติเหตุ","การรักษาก่อนหน้า","แพ้ยา","โรคประจำตัว",
  "ตั้งครรภ์","ผ่าตัด","วัคซีน","ประวัติครอบครัว",
  "ประวัติการสูบบุหรี่","ประวัติการดื่มแอลกอฮอล์",
  "previous treatment","prior treatment"
])
def to_canonical_th(token: str) -> str:
    t = (token or "").strip().lower()
    if not t: return ""
    if t in SYM_MAP.get("canonical", {}): t = SYM_MAP["canonical"][t]
    t = SYM_MAP.get("thai_alias", {}).get(t, t)
    t = re.sub(r"^(?:มี|เป็น|รู้สึก|รู้สึกว่า)\s*", "", t).strip()
    return t
def split_compounds(text: str):
    if not text: return []
    parts = re.split(r"\s*(?:,|\+|\||และ|กับ|ร่วมกับ)\s*", text.strip())
    out=[]
    for p in parts:
        p = re.sub(r"^(?:มี|เป็น|รู้สึก|รู้สึกว่า)\s*", "", p).strip()
        if not p: continue
        if p in {"น้ำมูกไหลไอ","ปวดหัวปวดท้ายทอย","คัดจมูกน้ำมูกไหล"}:
            mapping = {
                "น้ำมูกไหลไอ":["น้ำมูกไหล","ไอ"],
                "ปวดหัวปวดท้ายทอย":["ปวดหัว","ปวดท้ายทอย"],
                "คัดจมูกน้ำมูกไหล":["คัดจมูก","น้ำมูกไหล"]
            }[p]
            out.extend(mapping)
        else:
            out.append(p)
    return out
def normalize_list(tokens):
    out, seen = [], set()
    for tok in (tokens or []):
        for t in split_compounds(tok):
            t = to_canonical_th(t)
            if t and t not in STOP_TERMS and t not in seen:
                out.append(t); seen.add(t)
    return out
def normalize_free_text_multi(text): return normalize_list([text])

# 2) re-normalize
df["basket"]   = df["yes_raw"].apply(normalize_list)
df["no_norm"]  = df["no_raw"].apply(normalize_list)
df["idk_norm"] = df["idk_raw"].apply(normalize_list)
df = df[df["basket"].str.len() > 0].reset_index(drop=True)
df["search_norm"] = df["search_term"].apply(normalize_free_text_multi)

# 3) popularity (cold-start)
def _pop(series_of_lists):
    c = Counter()
    for lst in series_of_lists: c.update(lst)
    return c
global_pop = _pop(df["basket"])
by_seg = {seg: _pop(sub["basket"]) for seg, sub in df.groupby("seg")}

# 4) CF
transactions = df["basket"].tolist()
te = TransactionEncoder()
oht = te.fit_transform(transactions)
oht_df = pd.DataFrame(oht, columns=te.columns_)
X = oht.astype(np.float32)
SIM = cosine_similarity(X.T)
symptoms = list(te.columns_)
sym2idx = {s:i for i,s in enumerate(symptoms)}
def cf_score(selected, cand):
    idx = [sym2idx[s] for s in selected if s in sym2idx]
    if not idx or cand not in sym2idx: return 0.0
    j = sym2idx[cand]
    return float(SIM[j, idx].mean())

# 5) FP-Growth (1⇒1)
min_sup = max(5/len(df), 0.005)
freq = fpgrowth(oht_df, min_support=min_sup, use_colnames=True)
rules = association_rules(freq, metric="lift", min_threshold=1.0)
rules = rules[(rules["antecedents"].map(len)==1) & (rules["consequents"].map(len)==1)].copy()
rules["A"] = rules["antecedents"].apply(lambda s: next(iter(s)))
rules["B"] = rules["consequents"].apply(lambda s: next(iter(s)))
rules["rank_score"] = rules["lift"] * rules["confidence"]
rules = rules.sort_values(["rank_score","lift","confidence"], ascending=False)

rule_index = {}
for _, r in rules.iterrows():
    A, B = r["A"], r["B"]
    rule_index.setdefault(A, {})[B] = {
        "lift": float(r["lift"]),
        "confidence": float(r["confidence"]),
        "support": float(r["support"]),
    }

print(f"Rebuilt — rows={len(df)}, symptoms={len(symptoms)}, rules={len(rules)}")
print("Example search_norm →", df[['search_term','search_norm']].head(3).to_dict('records'))

Rebuilt — rows=991, symptoms=150, rules=36
Example search_norm → [{'search_term': 'มีเสมหะ, ไอ', 'search_norm': ['เสมหะ', 'ไอ']}, {'search_term': 'ไอ, น้ำมูกไหล', 'search_norm': ['ไอ', 'น้ำมูกไหล']}, {'search_term': 'ปวดท้อง', 'search_norm': ['ปวดท้อง']}]


In [ ]:
# re-run evaluation quietly after map rebuild
import contextlib, io, warnings, pandas as pd

buf = io.StringIO()
with contextlib.redirect_stderr(buf), warnings.catch_warnings():
    warnings.simplefilter("ignore", category=DeprecationWarning)
    eval_df = eval_all(alpha=ALPHA)

cols = ["Precision","Recall","MAP","Coverage"]
eval_df_display = eval_df.copy()
eval_df_display[cols] = eval_df_display[cols].round(6)
display(eval_df_display)

def pick(df, proto="LOO", K=5, metric="MAP"):
    row = df[(df["Proto"]==proto) & (df["K"]==K)]
    return (row[metric].iloc[0] if len(row) else float("nan"))

print(f"LOO MAP@5 = {pick(eval_df,'LOO',5,'MAP'):.3f}")
print(f"Beyond MAP@5 = {pick(eval_df,'Beyond',5,'MAP'):.3f}")
print(f"Coverage@5 = {pick(eval_df,'LOO',5,'Coverage'):.3f}")

,Proto,N,K,Precision,Recall,MAP,Coverage
0,LOO,440,3,0.284848,0.854545,0.656061,0.680000
1,LOO,440,5,0.187273,0.936364,0.674356,0.760000
2,LOO,440,7,0.138312,0.968182,0.679389,0.786667
3,LOO,440,10,0.099545,0.995455,0.682615,0.786667
4,Beyond,293,3,0.202503,0.512230,0.412211,0.566667
5,Beyond,293,5,0.148123,0.610068,0.436016,0.666667
6,Beyond,293,7,0.113603,0.654437,0.443492,0.733333
7,Beyond,293,10,0.085324,0.702218,0.449214,0.753333


LOO MAP@5 = 0.674
Beyond MAP@5 = 0.436
Coverage@5 = 0.760


In [ ]:
alphas = [0.5, 0.6, 0.7]
rows = []
for a in alphas:
    df_tmp = eval_all(alpha=a)
    v = df_tmp[(df_tmp["Proto"]=="Beyond") & (df_tmp["K"]==5)]["MAP"].iloc[0]
    rows.append({"alpha": a, "Beyond_MAP@5": float(v)})
import pandas as pd
grid_df = pd.DataFrame(rows)
display(grid_df.sort_values("Beyond_MAP@5", ascending=False).reset_index(drop=True))

,alpha,Beyond_MAP@5
0,0.5,0.436016
1,0.6,0.436016
2,0.7,0.436016


In [ ]:
from fastapi import FastAPI
from fastapi.responses import RedirectResponse, HTMLResponse
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

ALPHA = globals().get("ALPHA", 0.7)

class HealthResp(BaseModel):
    status: Literal["ok"]
    symptoms: int
    rules: int

class RecReq(BaseModel):
    age: Optional[int] = Field(None)
    sex: Literal["M","F","U"] = Field("U")
    selected: List[str] = Field(default_factory=list)
    top_k: int = Field(3, ge=1, le=20)

app = FastAPI(title="Agnos Symptom Recommender (Task 2)", version="1.0")

@app.get("/", include_in_schema=False)
def root():
    return RedirectResponse(url="/docs")

@app.get("/health", response_model=HealthResp)
def health():
    return HealthResp(status="ok", symptoms=len(symptoms), rules=len(rules))

@app.get("/app", response_class=HTMLResponse)
def app_ui():
    return """
    <html><body style="font-family: system-ui; line-height:1.5; padding:16px">
      <h2>Agnos Symptom Recommender</h2>
      <form method="post" action="/recommend" style="display:grid; gap:8px; max-width:520px">
        <label>Age <input name="age" type="number" /></label>
        <label>Sex
          <select name="sex"><option>M</option><option>F</option><option selected>U</option></select>
        </label>
        <label>Selected (comma-separated)
          <input name="selected" placeholder="ไอ, น้ำมูกไหล หรือ cough, runny nose"/>
        </label>
        <label>Top K <input name="top_k" type="number" value="3"/></label>
        <button type="submit">Recommend</button>
      </form>
      <p>ดูสเปก API ที่ <a href="/docs">/docs</a></p>
    </body></html>
    """

@app.post("/recommend")
def recommend_endpoint(req: RecReq):
    selected_norm, recs = recommend(age=req.age, sex=req.sex, selected=req.selected, top_k=req.top_k, alpha=ALPHA)
    return {
        "input": selected_norm,
        "alpha": ALPHA,
        "recommendations": recs,
        "disclaimer": "Not for diagnosis; triage assist only."
    }

print("FastAPI app ready → run uvicorn then open /docs")

FastAPI app ready → run uvicorn then open /docs


In [ ]:
import nest_asyncio, uvicorn, threading, time
nest_asyncio.apply()

def _run():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=_run, daemon=True)
server_thread.start()
time.sleep(1)
print("Uvicorn running on http://0.0.0.0:8000 (background)")

INFO:     Started server process [3449]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Uvicorn running on http://0.0.0.0:8000 (background)


In [ ]:
!pip -q install pyngrok~=7.0
from pyngrok import ngrok
import os

token = os.environ.get("XXX")
assert token, "Set NGROK_AUTHTOKEN env var first (..)"
ngrok.set_auth_token(token)

tunnel = ngrok.connect(addr=8000, proto="http")
print("Public URL:", tunnel.public_url)
print("Docs:", tunnel.public_url + "/docs")



Public URL: https://0b1c4da16508.ngrok-free.app
Open docs: https://0b1c4da16508.ngrok-free.app/docs
Health: https://0b1c4da16508.ngrok-free.app/health
INFO:     34.86.115.58:0 - "GET /health HTTP/1.1" 200 OK
GET /health -> 200 {"status":"ok","symptoms":150,"rules":36}
